In [1]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

#walletsList = pd.read_csv("../Common Wallet in Multiple Projects/pepe.csv")
#walletsList = pd.read_csv("../TokenAnalysis/testDEJITARUPARUGoodWallets.csv")
walletsList = pd.read_csv("ONEWALLETTEST.csv")
walletsList = walletsList.drop_duplicates()

df = pd.DataFrame(columns=list(['Wallet_Address','Tokens_Traded', 'Positive_Trades', 'Negative_Trades', 'Still_Holding_OR_Dusted', 'Sold_Incrementally_(NotJet)', 'Transfer_To_Other_Account_OR_NULL', 'Transfer_Stablecoin', 'Average_Profit']))
#df.to_csv('RESULTS_TEST_COWBYS.csv')

for i, row in walletsList[rowNum:].iterrows():
    tokensTraded = 0
    stillHoldingOrDusted = 0
    sellingIncrementally = 0
    accountTransferORNULL = 0
    stablecoinTransfer = 0
    profits = {}

    print("COUNT: ", count)
    bigDICT = {}
    #wallet = row["Address"]
    
    wallet = row["Wallet"]
    #print(f"CURRENT WALLET: {wallet}")

    positiveTrade = 0
    negativeTrade = 0

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        # STABLECOIN OR JUST TRANSFER
        if (sells - buys == 0.0 or excludeThisContract):
            continue

        # transfer maybe
        if buys == 0: continue

        profit = ((sells - buys) / buys) * 100.0
        
        if profit > 0: positiveTrade += 1
        else: negativeTrade += 1

        profits[contract] = profit
        tokensTraded += 1

    #CHANGE FILE NAME HERE
    with open("RESULTS_TEST_DONKEY_DETAILS.json", "a") as file:
        json.dump(profits, file)    
        
    profitsAVG = sum(profits.values())/len(profits)

    df.loc[0] = [wallet, tokensTraded, positiveTrade, negativeTrade, stillHoldingOrDusted, sellingIncrementally, accountTransferORNULL, stablecoinTransfer, profitsAVG]
    df.to_csv('RESULTS_TEST_DONKEY.csv', mode='a', index=False, header=False)

    count += 1



COUNT:  1


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
table = pd.read_csv("RESULTS_TEST_DONKEY.csv")
table.sort_values('Average_Profit', ascending=False)

KeyError: 'Average_Profit'